In [0]:
import requests
import json
import sys
from pyspark.sql.functions import lit
from pyspark.sql.functions import to_utc_timestamp, from_utc_timestamp, current_timestamp

In [0]:
# VARIÁVEIS
table_name = "ibge_brasil"

# CRIA TABELA
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS {table_name}
(sigla_estado    string     comment "Sigla do Estado (UF)"
,codigo_ibge     string     comment "Código do Estado no IBGE"
,nome_municipio  string     comment "Nome do Município"
,id_estado       long       comment "ID do Estado"
,nome_estado     string     comment "Nome do Estado"
,id_regiao       string     comment "ID da Região"
,regiao          string     comment "Nome da Região"
,sigla_regiao    string     comment "Sigla da Região"
,data_carga      timestamp  comment "Data da Carga"
)          
USING DELTA
PARTITIONED BY (sigla_estado)
COMMENT "
- Dados do IBGE. 
- Fonte: https://brasilapi.com.br"
"""
)

DataFrame[]

In [0]:
# ----------------------------------------------------------------------------
def municipios(siglaUF):
    # Retorna os municípios da unidade federativa (UF)
    try:
        # URL DE LOGIN
        url = f"https://brasilapi.com.br/api/ibge/municipios/v1/{siglaUF}?providers=dados-abertos-br,gov,wikipedia"

        headers = {
            "Content-Type": "application/json;charset=UTF-8",
        }
        response = requests.request("GET", url, headers=headers)
        if response.status_code != 200:
            sys.exit({response.status_code})
        return response.json()
    except Exception as e:
        raise e
# ----------------------------------------------------------------------------
def estados():
    # Retorna informações de todos estados do Brasil
    try:
        # URL DE LOGIN
        url = "https://brasilapi.com.br/api/ibge/uf/v1"

        headers = {
            "Content-Type": "application/json;charset=UTF-8",
        }
        response = requests.request("GET", url, headers=headers)
        if response.status_code != 200:
            sys.exit({response.status_code})
        return response.json()
    except Exception as e:
        raise e

In [0]:
# CRIA DATAFRAME COM A LISTAGEM DOS ESTADOS DO BRASIL
estado_df = spark.createDataFrame(estados())

# SELECIONA DOS ESTADOS DO BRASIL OS CAMPOS DE INTERESSE PADRONIZANDO OS NOMES 
estado_df = estado_df.selectExpr(
    'id as id_estado',
    'nome as nome_estado',
    'sigla as sigla_estado',
    'regiao.id as id_regiao',
    'regiao.nome as regiao',
    'regiao.sigla as sigla_regiao'
)

In [0]:
# CRIA DATAFRAME COM A LISTAGEM DOS MUNICIPIOS DO BRASIL
municipios_df = None

# REALIZA BUSCA DOS MUNICIPIOS POR SIGLA DO ESTADO (UF)
for sigla in [row['sigla_estado'] for row in estado_df.select('sigla_estado').collect()]:
    df_temp = spark.createDataFrame(municipios(sigla))
    df_temp = df_temp.selectExpr(
        'codigo_ibge',
        'nome as nome_municipio'
    )
    # ADICIONA A SIGLA DO ESTADO A CADA MUNICÍPIO
    df_temp = df_temp.withColumn('sigla_estado', lit(sigla))
    if municipios_df is None:
        municipios_df = df_temp
    else:
        municipios_df = municipios_df.unionByName(df_temp)

In [0]:
# CRIA DO DATAFRAME FINAL COM OS DADOS DOS MUNICIPIOS E ESTADOS DO BRASIL, ADICIONANDO A DATA DA CARGA
df_final = municipios_df.join(estado_df, on='sigla_estado')\
    .withColumn('data_carga', from_utc_timestamp(to_utc_timestamp(current_timestamp(), 'UTC'), 'America/Sao_Paulo'))

# ESCREVE OS DADOS NO BANCO DE DADOS
df_final.write.mode('overwrite').saveAsTable(table_name)

In [0]:
spark.sql(f'OPTIMIZE {table_name}').display()

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 27, null, null, 0, 0, 27, 27, true, 0, 0, 1757812117956, 1757812119359, 8, 0, null, List(0, 0), null, 9, 9, 0, 0, null)"
